In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [2]:
class OneToOneITU():
    def __init__(self, n, m, parameters = (None,None) , lbs=(0, 0)):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
     
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]
        
        self.max_A_j = np.max(parameters[0],0)

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, :]

    def get_V_ij(self, U_i, j_idx ,i_idx = None):
        if i_idx is None:
            return (self.B_ij[:,j_idx] -  U_i[:,None])/ self.A_ij[:,j_idx]
        else:
            return (self.B_ij[i_idx,j_idx] -  U_i[:,None])/ self.A_ij[i_idx,j_idx]

In [3]:
def generate_example(self, var_i, var_j, random_seed):
    np.random.seed(random_seed)
    k = 2

    θ_true = np.array([-0.1,0.1, 1,2])

    X_i = np.random.normal([5,var_i],[5,var_i], size = [self.n, k]).round(0)
    Y_j = np.random.normal([5,var_j],[5,var_j], size = [self.m, k]).round(0)
    #X_ijk = np.random.choice(10, size = (self.n_i,m_j, k))

    A = np.exp( np.abs((X_i[:,None,:]- Y_j[None,:,:]))@  θ_true[:k]).round(1)+.001
    B = np.abs((X_i[:,None,:]*Y_j[None,:,:])) @ θ_true[k:2*k] 
    print(len(np.unique(X_i)))
    self.A_ij = A
    self.B_ij = B
    self.max_A_j = np.max(A, axis= 0)

OneToOneITU.generate_example = generate_example

In [4]:
# def transpose_mkt(self):
#     n, m = self.n, self.m
#     self.m = n
#     self.n = m

#     self.A_ij = 1 / self.A_ij.T
#     self.B_ij = self.B_ij.T/ self.A_ij

# OneToOneITU.transpose_mkt = transpose_mkt

In [5]:
def check_primal_feas(self, mu_ij):
    print(f"i: { np.all(np.sum(mu_ij,axis=1) <= 1)}, j:  {np.all(np.sum(mu_ij,axis=0) <= 1) }")
    print(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}")

OneToOneITU.check_primal_feas = check_primal_feas

def check_IR(self,mu_ij,U_i,V_j, output = False):
    IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
    IR_j = np.sum((V_j - self.lb_V) * (1 - np.sum(mu_ij,axis=0))) 
    print(f"i: { IR_i}, j:  {IR_j }")
    if output is True:
        IR_i = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        IR_j = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return IR_i, IR_j
    
OneToOneITU.check_IR = check_IR

def check_CS(self,U_i,V_j, output = None):
    CS = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return CS, np.where(CS < 0 )
    print(f"min_ij D_ij : {(self.D_ij(U_i,V_j)).min()} ")
    
OneToOneITU.check_CS = check_CS 

def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    print("FEAS")
    self.check_primal_feas(mu_ij)
    print("________________")
    print("ε-CS")
    self.check_CS(U_i, V_j )
    print("________________")
    print("IR")
    self.check_IR(mu_ij, U_i, V_j )

OneToOneITU.check_all = check_all

def check_CS_i_or_j(self,u_i, v_j, side = 0):
    if side == 0:
        return u_i[:,None] - self.get_U_ij(v_j, np.arange(self.n))
    else:
        return v_j[None,:] - self.get_V_ij(u_i, np.arange(self.m))
    
OneToOneITU.check_CS_i_or_j = check_CS_i_or_j

# Auction algorithms

### Forward Auction

In [6]:
def forward_auction(self, data = None, tol_ε = None):
    if data is None:
        V_j = np.ones(self.m) * self.lb_V
        mu_i = np.ones( self.n, dtype= int) * (self.m +1)
    else:
        mu_ij_01 = data[0]
        V_j = data[1]
        id_i, id_j = np.where(mu_ij_01 > 0)
        mu_i = np.ones(self.n, dtype= int) * (self.m+1)
        mu_i[id_i] = id_j

    unassigned_i = np.where(mu_i == self.m + 1)[0] 
    n_unassigned_i = len(unassigned_i)
    iter = 0

    while n_unassigned_i > 0:
        iter += 1
        U_ij = self.get_U_ij( V_j, unassigned_i)
        perm_unmatched = np.all(U_ij <= self.lb_U+tol_ε, axis = 1)#np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_i[unassigned_i] += (self.m - mu_i[unassigned_i]) * perm_unmatched
        else:
            ### bidding phase
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned_i,1))), axis = 1)
            j_i = np.argmax(U_ij, axis= 1)
            masked_U_ij = np.where(j_i[:,None] == np.arange(self.m+1)[None, :], -np.inf,U_ij)
            w_i = np.max(masked_U_ij, axis=1)
            j_i_unique = np.unique(j_i)
            offers = np.where(j_i[:,None] == j_i_unique[None,:], 
                              self.get_V_ij(w_i- tol_ε,j_i_unique[None,:],unassigned_i[:,None]), 
                              np.nan)
       
            ### assignment phase
            i_j_among_unass = np.nanargmax(offers, axis = 0)
            i_j = unassigned_i[ i_j_among_unass]
            best_offer_j = offers[i_j_among_unass, np.arange(len(j_i_unique))]

            # modify solution
            mu_i[  np.any( mu_i[:,None] == j_i_unique , axis = 1) ] = self.m +1
            mu_i[i_j] = j_i_unique
            
            V_j[j_i_unique] = best_offer_j #- V_j[j_i_unique] + tol_ε / self.A_ij[i_j,j_i_unique] #np.maximum(best_offer_j - V_j[j_i_unique], tol_ε/self.A_ij[i_j,j_i_unique])
            # print(V_j)
        unassigned_i = np.where(mu_i == self.m + 1)[0]
        n_unassigned_i = len(unassigned_i)
        
    print(f"for: {iter}")
    matched_i =  mu_i < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_i[matched_i]]
    mu_ij_01 = mu_i[:,None] == np.arange(self.m +1)

    return mu_ij_01[:,:-1], U_i, V_j
    

OneToOneITU.forward_auction = forward_auction

#### TU

In [7]:
# n_TU = 25
# m_TU = 26
# A_TU = np.ones([n_TU,m_TU])
# B_TU = np.random.normal(40,5,[n_TU,m_TU]).round(0)

# example_TU = OneToOneITU( n_TU,m_TU, (A_TU,B_TU))
# mu_TU, u_TU, v_TU = example_TU.reverse_auction(tol_ε= 0.001)

In [8]:
# def solve_1to1(Φ_i_j):
#     n, m = np.shape(Φ_i_j)
#     M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
#                       [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
#     q = np.concatenate((np.ones(n), np.ones(m)))

#     model = grb.Model()
#     mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
#     model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
#     model.addConstr(M_z_a @ mu_a <= q)
    
#     model.Params.OutputFlag = 0
#     model.optimize()

#     return np.array(model.x, dtype= bool).reshape([n,m])*1

# mu_gurobi = solve_1to1(B_TU)
# print((mu_gurobi * B_TU).sum())
# print(u_TU.sum() +v_TU.sum())
# example_TU.check_all( (mu_TU, u_TU, v_TU ))

### Reverse Auction

In [9]:
def reverse_auction(self, data = None, tol_ε = None):
    if data is None:
        U_i = np.ones(self.n) * self.lb_U
        mu_j = np.ones(self.m, dtype= int) * (self.n +1)
    else:
        mu_j_01 = data[0]
        U_i = data[1]
        id_i, id_j = np.where(mu_j_01 >0)
        mu_j = np.ones(self.m, dtype= int) * (self.n+1)
        mu_j[id_j] = id_i

    unassigned_j = np.where(mu_j == self.n + 1)[0] 
    n_unassigned_j = len(unassigned_j)
    iter = 0

    while n_unassigned_j > 0:
        iter += 1
        V_ij = self.get_V_ij( U_i , unassigned_j)
        perm_unmatched = np.all(V_ij <= self.lb_V+ tol_ε, axis = 0) #np.all(V_ij <= self.lb_V, axis = 0)
        
        if np.any(perm_unmatched):
            mu_j[unassigned_j] += (self.n - mu_j[unassigned_j]) * perm_unmatched
        
        else:
            ### bidding phase
            V_ij = np.concatenate((V_ij, self.lb_V * np.ones((1,n_unassigned_j))), axis = 0)
            i_j = np.argmax(V_ij, axis= 0)
            masked_V_ij = np.where( np.arange(self.n+1)[:, None] == i_j[None,:] , -np.inf,V_ij)
            β_j = np.max(masked_V_ij, axis=0)
            i_j_unique = np.unique(i_j)
            offers = np.where(i_j_unique[:,None] == i_j[None,:], 
                              self.get_U_ij(β_j - tol_ε,i_j_unique[:,None] ,unassigned_j[None,:]), 
                              np.nan)

            ### assignment phase
            j_i_among_unass = np.nanargmax(offers, axis = 1)
            j_i = unassigned_j[j_i_among_unass]
            best_offer_i = offers[np.arange(len(i_j_unique)),j_i_among_unass]
            # modify solution
            mu_j[ np.any( mu_j[:,None] == i_j_unique , axis = 1) ] = self.n +1
            mu_j[j_i] = i_j_unique
            U_i[i_j_unique] = best_offer_i #- U_i[i_j_unique] + tol_ε * self.A_ij[i_j_unique,j_i]#np.maximum(best_offer_i - U_i[i_j_unique], tol_ε * self.A_ij[i_j_unique,j_i] )#/ self.max_A_j[j_i] )
                                        
        unassigned_j = np.where(mu_j == self.n + 1)[0]
        n_unassigned_j = len(unassigned_j)
        
    print(f"rev: {iter}") 
    matched_j =  mu_j < self.n
    V_j = np.ones(self.m) * self.lb_V
    V_j[mu_j < self.n] = self.get_V_ij(U_i, matched_j )[mu_j[matched_j],np.arange(matched_j.sum()) ]
    mu_j_01 = np.arange(self.n +1)[:,None] == mu_j[None,:]

    return mu_j_01[:-1,:], U_i, V_j
    

OneToOneITU.reverse_auction = reverse_auction

### Forward-backward auction

In [10]:
def drop_for_scaling(self, mu_ij_01, U_i,V_j, tol_ε, side = 1):
    if side == 1:
        violations_j = np.any( V_j[None,:] + tol_ε <  self.get_V_ij(U_i  ,np.arange(self.m)), axis=  0)
        #violations_j = np.any( V_j[None,:] <  self.get_V_ij(U_i + tol_ε ,np.arange(self.m)), axis=  0)
        # print(violations_j.sum())
        mu_ij_01[:,violations_j] = 0

        print(f"dropped: {violations_j.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

      
    else:
        violations_i = np.any( U_i[:,None] + tol_ε <  self.get_U_ij(V_j  ,np.arange(self.n)), axis=  1)
   
        mu_ij_01[violations_i,:] = 0

        print(f"dropped: {violations_i.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

    return mu_ij_01, U_i,V_j
OneToOneITU.drop_for_scaling = drop_for_scaling

In [11]:
def forward_backward_scaling(self, tol_ε ,tol_ε_obj, scaling_factor):
    mu_ij_01, u_i, v_j = self.forward_auction((np.zeros([self.n ,self.m]), np.ones(self.m) * self.lb_V),tol_ε = tol_ε)

    print("_________")
    while tol_ε > tol_ε_obj:
        # tol_ε *=  scaling_factor
        # print("####")
        # print( tol_ε)
        # mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε)
        mu_ij_01 = np.zeros([self.n ,self.m])
        mu_ij_01, u_i, v_j = self.reverse_auction((mu_ij_01, u_i), tol_ε= tol_ε)
        #self.check_IR(mu_ij_01, u_i,v_j)
        tol_ε *=  scaling_factor
        # mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε, side = 0)
        #mu_ij_01 = np.zeros([self.n ,self.m])
        mu_ij_01, u_i, v_j = self.forward_auction((mu_ij_01, v_j ), tol_ε = tol_ε)
        #self.check_CS(u_i,v_j)
        #self.check_IR(mu_ij_01, u_i,v_j)
        print("_________")   
        # if np.all(self.check_CS(u_i,v_j,True)[0]>= - tol_ε_obj):
        #     return mu_ij_01, u_i, v_j
    #tol_ε *=  scaling_factor
    # mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε)
    mu_ij_01 = np.zeros([self.n ,self.m])
    mu_ij_01, u_i, v_j = self.reverse_auction((mu_ij_01, u_i), tol_ε= tol_ε)
    mu_ij_01, u_i, v_j = self.forward_auction((mu_ij_01, v_j ), tol_ε = tol_ε)
    return mu_ij_01, u_i, v_j
OneToOneITU.forward_backward_scaling =forward_backward_scaling  

# Examples

In [12]:
# n_i =  500
# m_j  = 530
# example_mkt = OneToOneITU( n_i,m_j, lbs=(0,0))
# example_mkt.generate_example(2,2, random_seed = 1309887)

In [13]:
# example_mkt.A_ij = np.random.choice([.25,.5,1,2,4],size=[ n_i,m_j])

In [14]:
# example_mkt.A_ij.min()

In [15]:
# tol_ε_i = 10
# mu_ij_01, u_i, v_j = example_mkt.forward_auction((np.zeros([example_mkt.n ,example_mkt.m]), np.ones(example_mkt.m) *0),tol_ε = tol_ε_i)

In [16]:
# mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt.forward_backward_scaling(10,1e-12, 1/3)

In [17]:
# example_mkt.check_all((mu_ij_01_scaling, u_i_scaling, v_j_scaling))

In [18]:
# (u_i_scaling - example_mkt.lb_U ) [ np.where(mu_ij_01_scaling.sum(1)  == 0)]

In [38]:
# u_i_scaling.sum()+ v_j_scaling.sum()

In [85]:
n_i =  35
m_j  = 36
np.random.seed(123)
A_ =  np.random.choice([.25,.5,1,2,4], size= [n_i,1] ) * np.random.choice([.25,.5,1,2,4], size= [1, m_j] ) #  #np.random.choice([.25,.5,1,2,4], size= [n_i,m_j])
B_ = (np.random.randint(1,3, size= [ n_i,1]) * np.random.randint(1,3, size= [ 1,m_j]))**2 +1 #np.random.choice([200, 300], size=(n_i, m_j), p=[.9,.1 ]) * 100#np.random.randint(n_i/2,n_i,size = [n_i,m_j]) # np.random.choice([0,1], size= [n_i,m_j])*10000
example_mkt_2 = OneToOneITU(  n_i,m_j,parameters=(A_,B_))

In [86]:
A_.mean()

2.7186507936507938

In [90]:
mu_ij_01_noscaling2, u_i_noscaling2, v_j_noscaling2 = example_mkt_2.forward_auction(tol_ε= .001)
example_mkt_2.check_all((mu_ij_01_noscaling2, u_i_noscaling2, v_j_noscaling2))

for: 60704
FEAS
i: True, j:  True
#matched: 35 over 35
________________
ε-CS
min_ij D_ij : -0.0009411764705860418 
________________
IR
i: 0.0, j:  0.0


In [65]:
mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt_2.forward_backward_scaling(1,1e-8, 1/4)

for: 737
_________
rev: 471
for: 215
_________
rev: 339
for: 120
_________
rev: 342
for: 116
_________
rev: 367
for: 106
_________
rev: 404
for: 107
_________
rev: 366
for: 111
_________
rev: 397
for: 104
_________
rev: 363
for: 133
_________
rev: 357
for: 108
_________
rev: 394
for: 110
_________
rev: 358
for: 107
_________
rev: 404
for: 105
_________
rev: 394
for: 132
_________
rev: 342
for: 111
_________
rev: 382
for: 110


In [66]:
example_mkt_2.check_all((mu_ij_01_scaling, u_i_scaling, v_j_scaling))

FEAS
i: True, j:  True
#matched: 350 over 350
________________
ε-CS
min_ij D_ij : -3.506155575022978e-09 
________________
IR
i: 0.0, j:  0.0


In [36]:
np.min(u_i_scaling[:,None] - example_mkt_2.get_U_ij(v_j_scaling, np.arange(n_i)))


-9.094947017729282e-12

In [37]:
np.min(v_j_scaling[None,:] - example_mkt_2.get_V_ij(u_i_scaling, np.arange(m_j)))

-3.637978807091713e-11

In [46]:
np.all(mu_ij_01_scaling.sum(1) <=1)

True

In [47]:
np.sum((u_i_scaling - example_mkt_2.lb_U ) * (mu_ij_01_scaling.sum(1) - 1))

0.0

In [48]:
np.sum((v_j_scaling - example_mkt_2.lb_V ) * (mu_ij_01_scaling.sum(0) - 1))

0.0

To do:
* try nasty example with B binary times a constant
* try to scale also i forward auction
* try convex taxes

Next steps:
* $ij$-specific scaling
* max iter in the backward iteration
* (IMPORTANT) now all you are computing V_ij for all i and j in backward

### Standard scaling

In [134]:
n_i =  m_j  = 1000

np.random.seed(12993)
A_ =  np.ones([n_i,m_j]) #np.random.choice([.25,.5,1,2,4], size= [n_i,1] ) * np.random.choice([.25,.5,1,2,4], size= [1, m_j] ) #  #np.random.choice([.25,.5,1,2,4], size= [n_i,m_j])
B_ = (np.random.randint(1,20, size= [ n_i,1]) * np.random.randint(1,20, size= [ 1,m_j]))  #np.random.choice([200, 300], size=(n_i, m_j), p=[.9,.1 ]) * 100#np.random.randint(n_i/2,n_i,size = [n_i,m_j]) # np.random.choice([0,1], size= [n_i,m_j])*10000
example_mkt_n_by_n = OneToOneITU(  n_i,m_j,parameters=(A_,B_), lbs=( - np.inf, 0))

In [135]:
B_.max()

361

In [145]:
tol_ε = 1000
mu, u, v = example_mkt_n_by_n.forward_auction(tol_ε = tol_ε)
# example_mkt_n_by_n.check_all((mu,u,v))


for: 1000


In [146]:
# mu, u, v = example_mkt_n_by_n.forward_auction((np.zeros([n_i,n_i]), v),tol_ε = 200)


In [138]:
def Bertsekas_scaling(self,tol_ε_initial,tol_ε_obj, scaling_factor):
    tol_ε = tol_ε_initial
    mu_ij, u_i, v_j = self.forward_auction(tol_ε = tol_ε)
    while tol_ε > tol_ε_obj:
        tol_ε /= scaling_factor
        mu_ij, u_i, v_j = self.forward_auction((np.zeros([self.n,self.m]), v_j),tol_ε = tol_ε)

    return mu_ij, u_i, v_j

OneToOneITU.Bertsekas_scaling =Bertsekas_scaling

In [139]:
mu_B_scaling, u_B_scaling, v_B_scaling = example_mkt_n_by_n.Bertsekas_scaling(100,1e-10,2)

for: 1000
for: 774
for: 835
for: 983
for: 974
for: 979
for: 1417
for: 1407
for: 1366
for: 1747
for: 1149
for: 1600
for: 1272
for: 2323
for: 1614
for: 1647
for: 569
for: 956
for: 829
for: 1974
for: 1557
for: 1299
for: 1201
for: 1228
for: 872
for: 1211
for: 1302
for: 1449
for: 1157
for: 703
for: 1008
for: 959
for: 834
for: 973
for: 1577
for: 952
for: 894
for: 575
for: 1116
for: 570
for: 429


In [140]:
example_mkt_n_by_n.check_all((mu_B_scaling, u_B_scaling, v_B_scaling))

FEAS
i: True, j:  True
#matched: 1000 over 1000
________________
ε-CS
min_ij D_ij : -4.547473508864641e-11 
________________
IR
i: nan, j:  0.0


/var/folders/9x/rx50bd3s64g0kvsl05kqvl000000gn/T/ipykernel_42716/2722923942.py:8: RuntimeWarning: invalid value encountered in multiply
  IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))


In [141]:
u_B_scaling.min()

-310.7942135507292

In [142]:
v_scaling_2.min()

9.094947017729282e-11

In [143]:
example_mkt_n_by_n.lb_U = 0
mu_scaling_2, u_scaling_2, v_scaling_2 = example_mkt_n_by_n.forward_backward_scaling(100,1e-10,1/2)
example_mkt_n_by_n.lb_U = -np.inf

for: 614
_________
rev: 190
for: 322
_________
rev: 151
for: 227
_________
rev: 134
for: 278
_________
rev: 199
for: 375
_________
rev: 196
for: 477
_________
rev: 278
for: 871
_________
rev: 432
for: 646
_________
rev: 161
for: 1244
_________
rev: 381
for: 1019
_________
rev: 756
for: 1558
_________
rev: 688
for: 791
_________
rev: 452
for: 803
_________
rev: 336
for: 1233
_________
rev: 628
for: 990
_________
rev: 233
for: 724
_________
rev: 533
for: 765
_________
rev: 264
for: 977
_________
rev: 310
for: 548
_________
rev: 533
for: 1090
_________
rev: 353
for: 600
_________
rev: 187
for: 676
_________
rev: 347
for: 701
_________
rev: 119
for: 73
_________
rev: 264
for: 1111
_________
rev: 261
for: 1102
_________
rev: 592
for: 697
_________
rev: 145
for: 463
_________
rev: 297
for: 956
_________
rev: 575
for: 728
_________
rev: 170
for: 435
_________
rev: 175
for: 400
_________
rev: 487
for: 933
_________
rev: 417
for: 1232
_________
rev: 466
for: 896
_________
rev: 143
for: 556
____

In [147]:
example_mkt_n_by_n.check_all((mu_scaling_2, u_scaling_2, v_scaling_2 ))

FEAS
i: True, j:  True
#matched: 1000 over 1000
________________
ε-CS
min_ij D_ij : -4.547473508864641e-11 
________________
IR
i: nan, j:  0.0


/var/folders/9x/rx50bd3s64g0kvsl05kqvl000000gn/T/ipykernel_42716/2722923942.py:8: RuntimeWarning: invalid value encountered in multiply
  IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
